In [ ]:
import numpy as np
import pandas as pd 
from torchsummary import summary    
import os
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF
from tqdm import tqdm
import torch.optim as optim
from PIL import Image
from torch.utils.data import Dataset
import random
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import PIL.Image as Image
import os
from sklearn.metrics import jaccard_score, f1_score
# import wandb
import torchvision
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import albumentations as A
from albumentations.pytorch import ToTensorV2
from scipy.spatial.distance import directed_hausdorff
import re

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

In [ ]:
model = torch.hub.load('pytorch/vision:v0.8.0', 'deeplabv3_resnet50', pretrained=True)

# Modify the last convolutional layer
model.classifier[4] = nn.Conv2d(in_channels=256, out_channels=1, kernel_size=(1,1))

# If there's an auxiliary classifier, modify it similarly
if model.aux_classifier is not None:
    model.aux_classifier[4] = nn.Conv2d(in_channels=256, out_channels=1, kernel_size=(1,1))

# load weights into the model 
model.load_state_dict(torch.load('deeplabv3_leukemia.pth'))
model.to(device)

In [ ]:
def pad_to_aspect_ratio(image, output_size, pad_value=0):
  """
  Pads an image to a specific output size while maintaining aspect ratio.

  Args:
      image: PIL image object.
      output_size: Tuple of (height, width) for the desired output size.
      pad_value: The value to use for padding (default: 0).

  Returns:
      A padded PIL image object.
  """
  w, h = image.size

  target_ratio = output_size[0] / output_size[1]
  image_ratio = w / h

  if image_ratio > target_ratio:
    # Pad on top and bottom
    new_h = int(w / target_ratio)
    padding_top = (new_h - h) // 2
    padding_bottom = new_h - h - padding_top
  else:
    # Pad on left and right
    new_w = int(h * target_ratio)
    padding_left = (new_w - w) // 2
    padding_right = new_w - w - padding_left

  padded_image = F.pad(image, (padding_left, padding_right, padding_top, padding_bottom), value=pad_value)
  return padded_image


In [ ]:
# Transformation for the input images
resize = 448
transform = A.Compose(
    [
        # A.Resize(height=resize, width=resize),
        A.Lambda(lambda x: pad_to_aspect_ratio(x, (resize, resize))),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ])

# Directories
input_folder = '/home/dawlat.akaila/Documents/DL_LABS/PROJECT/try'
output_folder = '/home/dawlat.akaila/Documents/DL_LABS/PROJECT/try_masks'
os.makedirs(output_folder, exist_ok=True)

# Predict masks for images in the input folder
with torch.no_grad():
    for filename in os.listdir(input_folder):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.tif')):
            img_path = os.path.join(input_folder, filename)
            image = Image.open(img_path).convert("RGB")
            image = np.array(image)

            augmented = transform(image=image)
            input_tensor = augmented["image"].unsqueeze(0).to(device)

            output = model(input_tensor)['out']
            predicted_mask = torch.sigmoid(output).squeeze().cpu().numpy()

            # Convert the predicted mask to binary mask
            binary_mask = (predicted_mask > 0.5).astype(np.uint8) * 255

            # Save the predicted mask
            mask_image = Image.fromarray(binary_mask)
            mask_image.save(os.path.join(output_folder, filename))

print("Mask prediction completed. Masks saved in:", output_folder)